In [36]:
import pandas as pd
import numpy as np
import os 

In [37]:
#os.chdir('/home/s212945/snap/snapd-desktop-integration/current/Documents/Thesis/MSc_PopSyn/Sigga_Luis/Data/WGAN/V1')

In [38]:
df_dtypes = {
    'CarModelYear': 'category',
    'Education': 'category',
    'FamNumAdults': 'int64',
    'FamNumPers': 'int64',
    'FuelType': 'category',
    'Gender': 'category',
    'Handicap': 'category',
    'HomeAdrZone': 'category',
    'HousehCarOwnership': 'category',
    'HousehNumAdults': 'int64',
    'HousehNumPers': 'int64',
    'HousehNumcars': 'int64',
    'HwDayspW': 'category',
    'IncFamily2000': 'int64',
    'IncRespondent2000': 'int64',
    'KidsBetween0and4': 'int64',
    'KidsBetween4and15': 'int64',
    'MunicipalityDest': 'category',
    'MunicipalityOrigin': 'category',
    'PopSocio': 'category',
    'PrimOccZone': 'category',
    'RespAgeCorrect': 'int64',
    'RespHasBicycle': 'category',
    'RespHasRejsekort': 'category',
    'RespIsmemCarshare': 'category',
    'RespPrimOcc': 'category',
    'ResphasDrivlic': 'category',
    'Sector': 'category',
    'WorkHourType': 'category',
    'WorkHoursPw': 'float64',
    'Year': 'category',
    'HomeParkPoss': 'category',
    'HousehAccomodation': 'category',
    'HousehAccOwnorRent': 'category',
    'PosInFamily': 'category',
    'PrimModeDay': 'category',
    'ModeChainTypeDay': 'category',
    'RespHasSeasonticket': 'category',
    'MissIncome': 'category'
}
#readfile
df = pd.read_csv('DatGan_Population.csv', dtype=df_dtypes)

In [39]:
df.drop(df.columns[df.columns.str.contains('unnamed', case=False)], axis=1, inplace=True)

In [40]:
df['HousehCarOwnership'].unique()

['1.0', '2.0', '0.0', '3.0', '4.0', ..., '10.0', '20.0', '7.0', '9.0', '8.0']
Length: 14
Categories (14, object): ['0.0', '1.0', '10.0', '11.0', ..., '6.0', '7.0', '8.0', '9.0']

In [41]:
df.columns

Index(['FamNumPers', 'HousehNumAdults', 'IncFamily2000', 'WorkHoursPw',
       'FamNumAdults', 'HousehNumPers', 'HousehCarOwnership',
       'IncRespondent2000', 'HousehNumcars', 'Year', 'Gender',
       'RespAgeCorrect', 'Education', 'Handicap', 'PopSocio', 'RespPrimOcc',
       'KidsBetween0and15', 'KidsBetween0and4', 'CarModelYear',
       'MunicipalityOrigin', 'MunicipalityDest', 'HomeAdrZone', 'PrimOccZone',
       'WorkHourType', 'RespHasBicycle', 'ResphasDrivlic', 'RespHasRejsekort',
       'RespIsmemCarshare', 'Sector', 'HwDayspW', 'FuelType', 'HomeParkPoss',
       'RespHasSeasonticket', 'HousehAccomodation', 'HousehAccOwnorRent',
       'PosInFamily', 'PrimModeDay', 'ModeChainTypeDay', 'MissIncome'],
      dtype='object')

In [42]:
simMobSamples = df.copy()

In [43]:
#attributes that are the same 
simMobSamples.rename(columns={"IncRespondent2000": "Income_SCALED", "size":"HousehNumPers", "HomeAdrZone": "home_address_id", "PrimOccZone":"activity_address_id", "CarModelYear":"Vehicle_model", "HousehCarOwnership":"Vehicle_category_id",
                               "FuelType":"vehicle_drivetrain", "adult":"HousehNumAdults", "child_under4":"KidsBetween0and4", "child_under15":"KidsBetween0and15", "IncFamily2000":"Income_FAM_SCALED", "MissIncome":"missing_income"}, inplace=True)


simMobSamples.Income_SCALED.clip(lower=0, inplace=True)
simMobSamples.Income_FAM_SCALED.clip(lower=0, inplace=True)

#Gender
#change encoding
simMobSamples['gender_id'] = simMobSamples.Gender.cat.rename_categories({'1': '0', '2': '1'})

#missing_income
#binary
#if income is 0 then missing
#car_license
#binary
simMobSamples['car_licence'] = np.where(simMobSamples.ResphasDrivlic == '1.0', 1, 0) 

#is_student
#binary
simMobSamples['is_student'] = np.where(simMobSamples.PopSocio=='0',1 ,0)

#time_restriction
#binary
simMobSamples['time_restriction'] = np.where((simMobSamples.WorkHourType ==1) | (simMobSamples.WorkHourType==2), 1,0)

#work at home
#binary
#if person works 5,6,7 days at home per week then works from home
simMobSamples['work_at_home'] = np.where((simMobSamples.HwDayspW == '5.0') | (simMobSamples.HwDayspW == '6.0')|(simMobSamples.HwDayspW == '7.0'), 1,0)

#fixed workplace
#binary
#if person has an activity addres then they have fixed workplace
simMobSamples['fixed_workplace'] = np.where(simMobSamples['activity_address_id'] == 'MISSING', 0,1)

#vehicle id
simMobSamples['vehicle_id'] = simMobSamples['HousehNumcars']

#education_type_id
#not sutdents have ''
#other student defined as students that have other encodings in RespPrimOcc but is_student is 1
simMobSamples['education_type_id'] = ''
simMobSamples.loc[simMobSamples['is_student'] == 1, 'education_type_id'] = 6 #other student
simMobSamples.loc[simMobSamples['is_student'] == 0, 'education_type_id'] = 0 #not a student
simMobSamples.loc[simMobSamples['RespPrimOcc'] == '103.0', 'education_type_id'] = 1 #preschool_student
simMobSamples.loc[simMobSamples['RespPrimOcc'] == '107.0', 'education_type_id'] = 2 #priamryschool_student 
simMobSamples.loc[simMobSamples['RespPrimOcc'] == '116.0', 'education_type_id'] = 3 #highschool_student 
simMobSamples.loc[simMobSamples['RespPrimOcc'] == '130.0', 'education_type_id'] = 4 #trainee 
simMobSamples.loc[simMobSamples['RespPrimOcc'] == '120.0', 'education_type_id'] = 5 #universityStudent


#Person_type_id
#cannot interpret parttime
#students have ''
#RespPrimOcc encoding from TU data
simMobSamples['Person_type_id'] = ''
simMobSamples.loc[simMobSamples['education_type_id'] != 0 , 'Person_type_id'] = 0 #student
simMobSamples.loc[(simMobSamples['RespPrimOcc'].isin(['210.0','221.0','211.0'])), 'Person_type_id'] = 1 #fulltime
simMobSamples.loc[simMobSamples['WorkHoursPw']<37, 'Person_type_id'] = 2 #parttime
simMobSamples.loc[simMobSamples['RespPrimOcc'].isin(['390.0','222.0']), 'Person_type_id'] = 3 #homemaker
simMobSamples.loc[simMobSamples['RespPrimOcc'] == '310.0', 'Person_type_id'] = 5 #unemployed
simMobSamples.loc[simMobSamples['RespPrimOcc'].isin(['360.0','370.0']), 'Person_type_id'] = 4 #retired
simMobSamples.loc[simMobSamples['RespPrimOcc'].isin(['320.0','350.0']), 'Person_type_id'] = 6 #disabled 
simMobSamples.loc[simMobSamples['RespPrimOcc'].isin(['231.0','232.0','233.0']), 'Person_type_id'] = 7 #leave

#age_category_id
#Define age bins and corresponding categories
bins = [0, 18, 25, 30, 50, 65, max(simMobSamples.RespAgeCorrect)]
cats = [3,4,5,6,7,8]
simMobSamples['age_category_id'] = pd.cut(simMobSamples.RespAgeCorrect,bins,labels=cats,right=False, include_lowest=True)

#public_transport_pass
#if person has some kind of rejsekort
simMobSamples['public_transport_pass'] = np.where(simMobSamples.RespHasRejsekort == '2.0', 0,1)

In [44]:
simMobSamples.head(100)

,FamNumPers,HousehNumAdults,Income_FAM_SCALED,WorkHoursPw,FamNumAdults,HousehNumPers,Vehicle_category_id,Income_SCALED,HousehNumcars,Year,...,car_licence,is_student,time_restriction,work_at_home,fixed_workplace,vehicle_id,education_type_id,Person_type_id,age_category_id,public_transport_pass
0,3,2,905,36.0,2,1,1.0,422,2,2019,...,1,0,0,1,1,2,0,2,3,0
1,4,2,841,37.0,2,2,2.0,422,2,2012,...,1,0,0,1,1,2,0,1,4,1
2,4,2,912,0.0,2,2,0.0,417,1,2016,...,1,1,0,1,0,1,3,2,8,1
3,4,2,190,15.0,2,1,2.0,422,1,2009,...,1,0,0,0,1,1,0,2,6,0
4,2,2,883,37.0,2,3,0.0,1333,1,2014,...,0,0,0,1,1,1,5,0,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5,2,913,0.0,2,4,2.0,418,1,2021,...,0,0,0,0,1,1,2,2,6,0
96,1,2,913,37.0,2,4,0.0,422,0,2021,...,1,0,0,1,1,0,0,1,7,0
97,2,2,913,37.0,3,3,1.0,422,3,2009,...,1,1,0,1,0,3,5,0,7,1
98,3,1,704,50.0,3,4,3.0,422,1,2010,...,1,0,0,0,1,1,0,1,3,0


In [45]:
final = simMobSamples[["Income_SCALED", "HousehNumPers", "home_address_id", "activity_address_id", "Vehicle_model", "Vehicle_category_id",
                               "vehicle_drivetrain", "HousehNumAdults", "KidsBetween0and4", "KidsBetween0and15", "Income_FAM_SCALED", 'gender_id', 'missing_income',
                                'car_licence', 'is_student', 'time_restriction', 
                                'work_at_home', 'vehicle_id', 'fixed_workplace', 'Person_type_id',
                                'education_type_id', 'age_category_id', 'public_transport_pass']]

In [46]:
final.head(1000)

,Income_SCALED,HousehNumPers,home_address_id,activity_address_id,Vehicle_model,Vehicle_category_id,vehicle_drivetrain,HousehNumAdults,KidsBetween0and4,KidsBetween0and15,...,car_licence,is_student,time_restriction,work_at_home,vehicle_id,fixed_workplace,Person_type_id,education_type_id,age_category_id,public_transport_pass
0,422,1,265212.0,510055.0,2000.0,1.0,1.0,2,0,0,...,1,0,0,1,2,1,2,0,3,0
1,422,2,430164.0,751322.0,2000.0,2.0,2.0,2,0,0,...,1,0,0,1,2,1,1,0,4,1
2,417,2,3996.0,MISSING,2006.0,0.0,1.0,2,0,0,...,1,1,0,1,1,0,2,3,8,1
3,422,1,1308.0,787135.0,2012.0,2.0,1.0,2,0,3,...,1,0,0,0,1,1,2,0,6,0
4,1333,3,707094.0,810125.0,2007.0,0.0,1.0,2,0,1,...,0,0,0,1,1,1,0,5,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,419,4,2008.0,760235.0,1998.0,2.0,1.0,2,1,0,...,0,0,0,0,1,1,5,0,6,1
996,303,1,3707.0,607065.0,2007.0,1.0,NoCar,1,0,0,...,0,1,0,1,0,1,2,6,3,0
997,388,1,746062.0,810223.0,1999.0,1.0,2.0,2,0,0,...,1,0,0,1,1,1,1,0,6,1
998,407,2,740222.0,MISSING,2004.0,1.0,1.0,2,2,0,...,1,0,0,0,2,0,6,0,3,0


In [50]:
final.to_csv('encoded_samples_simMob_Datgan.csv', index=False)

In [48]:
final.Income_SCALED.loc[final.Income_SCALED<0]

Series([], Name: Income_SCALED, dtype: int64)

In [49]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)